In [ ]:
import pandas as pd
import numpy as np
import cv2
import json
import os
import matplotlib.pyplot as plt
import random
import seaborn as sns
from keras.models import Sequential
from keras import optimizers
from keras import backend as K
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import ipynb.fs.full.helper as h

from imp import reload
reload(h)

In [ ]:
directory = h.directory
OC_Net = h.OC_Net
df = h.df
image_directory = h.image_directory

df.head(10)

In [ ]:
data = []
img_size = 224
mask_describe = ['face_with_mask',"face_no_mask"]
labels={'mask':0,'without mask':1}
type_mask = []
fileste= ".json"

def get_bounding(j,i,data,labels_item):
    x,y,w,h = j["BoundingBox"]
    img = cv2.imread(os.path.join(image_directory,i),1)
    img = img[y:h,x:w]
    img = cv2.resize(img,(img_size,img_size))
    data.append([img,labels_item])
    
def get_json_file(PathName):
    with open(PathName,'r') as f:
        return json.load(f)

for i in df["name"].unique():
    f = i+fileste
    for j in get_json_file(os.path.join(directory,f)).get("Annotations"):
        if j["classname"] in mask_describe[0]:
            get_bounding(j,i,data,labels['mask'])
            type_mask.append(labels['mask'])
            
        if j["classname"] in mask_describe[1]:
            get_bounding(j,i,data,labels['without mask'])
            type_mask.append(labels['without mask'])

random.shuffle(data)  

In [ ]:
sns.countplot(type_mask)

In [ ]:
features = []
labels = []
for f,l in data:
    features.append(f)
    labels.append(l)

In [ ]:
features[0].shape

In [ ]:
features = np.array(features)/255.0
features = features.reshape(-1,img_size,img_size,3)
labels = np.array(labels)

print(np.unique(labels))
print(features.shape)

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2

base_model_mobile = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224,224,3))

for layer in base_model_mobile.layers:
    layer.trainable = False
    
model_mobile = Sequential()
model_mobile.add(base_model_mobile)
model_mobile.add(Flatten())
model_mobile.add(Dense(1,activation='sigmoid'))
model_mobile.summary()

In [ ]:
train_f,val_f,train_l,val_l=train_test_split(features, labels,train_size=0.8,random_state=0)
datagen = ImageDataGenerator(
        rotation_range= 17,    
        width_shift_range=0.1,
        height_shift_range=0.1,  
        zca_epsilon = 1e-05, 
        horizontal_flip=True, 
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.1,
        brightness_range = [0.1, 10], 
)
datagen.fit(train_f)

In [ ]:
model_mobile.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=['accuracy'])

history_mobile = model_mobile.fit(datagen.flow(train_f, train_l, batch_size=32),
                   steps_per_epoch=train_f.shape[0]//32,
                   epochs=15,
                   verbose=1,
                   validation_data=(val_f, val_l))


In [ ]:
history_list = []
history_list.append(history_mobile)

In [ ]:
model_mobile.evaluate(val_f, val_l)
prediction_mobile = model_mobile.predict(val_f)

In [ ]:
model_mobile.save(h.modelsVMask)

In [ ]:
import numpy
import pandas 
import matplotlib.pyplot as plt
%matplotlib inline

#1 
fig,ax = plt.subplots(figsize=(22,20))

print(ax)
ax7 = fig.add_subplot(1,2,1)
ax8 = fig.add_subplot(1,2,2)

#3
ax7.plot(history_mobile.history['accuracy'])
ax7.plot(history_mobile.history['val_accuracy'])
ax7.set_title('mobileNet Training accuracy vs Validation accuracy')
ax7.set_ylabel('accuracy')
ax7.set_xlabel('Epoch')
ax7.legend(['Train', 'Validation'], loc='upper left')

ax8.plot(history_mobile.history['loss'])
ax8.plot(history_mobile.history['val_loss'])
ax8.set_title('mobileNet Training Loss vs Validation Loss')
ax8.set_ylabel('Loss')
ax8.set_xlabel('Epoch')
ax8.legend(['Train', 'Validation'], loc='upper left')

In [ ]:
def extract_face(face):
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array

def get_label(frame,img_size):
    im = cv2.resize(frame,(img_size,img_size))
    im = np.array(im)/255.0
    im = im.reshape(1,img_size,img_size,3)
    result = model.predict(im)
    label_Y=1 if result>0.3 else 0
    return label_Y,result

In [ ]:
# 4 images to test and display
test_img = ["0015.jpg","0834.jpg","0675.jpg","1035.jpeg"]
image_test_directory = h.image_test_directory

In [ ]:
model = model_mobile

# adjust image darker and light 
adjust_light,rows,cols = 2.2,3,2
figure= plt.figure(figsize = (14,14))
axes = []
assign = {'0':'Mask','1':"No Mask"}
face_list = [] # save no mask

for j,im in enumerate(test_img):
    image =  cv2.imread(os.path.join(image_test_directory,im),1)
    back_image = image
    image = cv2.LUT(image.astype(np.uint8), np.array([((i / 255.0) ** (1.0 / adjust_light)) * 255 for i in np.arange(0, 256)]).astype(np.uint8))
    blob = cv2.dnn.blobFromImage(image=cv2.resize(image, (300,300)), scalefactor=1.0, size=(300, 300), mean=(104.0, 177.0, 123.0))
    
    OC_Net.setInput(blob)
    detections = OC_Net.forward()
    length = detections.shape[2]
    (hight, width) = image.shape[:2]
    for i in range(0, length):
        # get the face box
        boxs = detections[0, 0, i, 3:7] * np.array([width, hight, width, hight])
        (startX, startY, endX, endY) = boxs.astype("int")
        frame = image[startY:endY, startX:endX]
        
        # according to the confidence_threshold, to get the label
        confidence_threshold = detections[0, 0, i, 2]
        if confidence_threshold > 0.4:
            label_Y,result = get_label(frame,img_size)
            print(label_Y,result)
           
            # add the Text
            cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
            cv2.putText(image,assign[str(label_Y)] + "{: .2f}".format(float(result)) , (startX, startY-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (36,255,12), 2)
            
            ## save nomask people mesage
            if(label_Y==1):
                print("1")
#                     plt.imshow(back_image)
                face_list.append(im)
                print(im)

    axes.append(figure.add_subplot(rows, cols, j+1))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()